In [ ]:
import numpy as np 
import pandas as pd
from sklearn import datasets
import seaborn as sns
from sklearn.covariance import EllipticEnvelope
from sklearn.datasets import make_blobs
from sklearn import preprocessing

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Датасет титаник для проверок

In [ ]:
df = pd.read_csv('/kaggle/input/titanic/train.csv')

Датасет M5 для проверок

In [ ]:
df = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/calendar.csv')

# Работа со временем

**Функция приведения значений unixtime → datetime**

In [ ]:
#Входные данные - колонка в unixtime для датафрейма с названием df.
#Выходные данные - в df колонка из unixtime преобразуется в datetime.
def unixtime_to_datetime(unixtime_colomn):
    df[unixtime_colomn] = pd.to_datetime(df[unixtime_colomn], unit='s')

**Разбиваем дату на год/месяц/день недели (число 1-7 + название дня)**

In [ ]:
def datetime_into_colums(column): #На входе передаётся колонка datetime, над которой производится преобразование в год/месяц/день
    
    df['column'] = pd.to_datetime(df['column'],format='%d-%m-%Y %H:%M')

    df['year']=df['column'].dt.year 
    df['month']=df['column'].dt.month 
    df['day']=df['column'].dt.day
    df['dayofweek_num']=df['column'].dt.dayofweek  
    df['dayofweek_name']=df['column'].dt.weekday_name

Также обработать данные типа datetime можно наиболее универсальным способом используя парсер из библиотеки dateutil. Она позволяет передавать на вход парсеру дату в любом формате и переводить в один формат.
Пример кода ниже:

In [ ]:
from dateutil import parser

date = parser.parse('2011/12/15')
# Это также можно использовать для всех данных

# Заполнение числовых пропусков в таблице

**Выделение null-значений в датафрейме**

На вход передаётся тип вставки значения ('value', 'mean', 'equal'), который отвечает за подстановку значения (см.комментарии)

In [43]:
def fill_value (type_insert, column_na, groupby_parametr): 
        #Выделение null-значений в датафрейме
    if type_insert=='value':
        df[column_na] = df[column_na].fillna(-999) #пустые значения можно выделить значением, которое не встречается в df, таким образом мы указываем, что в данном значении раньше был null
        #Заполнение null-значений средним значением из имеющихся
    if type_insert=='mean':
        df[column_na] = df.groupby(groupby_parameter)[column_na].transform(
        lambda grp: grp.fillna(np.mean(grp))
    )
        #Замена null-значений уже существующими значениями
    if type_insert=='equal':
        df[column_na] = df[column_na].ffill().bfill()

fill_value('value', 'Age', '')
df

# Работа с выбросами, значениями нетипичными для датафрейма (ошибками в источнике данных)

In [ ]:
Построение графиков доступно в библиотеке seaborn

In [ ]:
sns.boxplot(x=df['Age'])

**2 страдегии работы с выбросами**

1 - отбросить строки с выбросами

//q = df['Age'].quantile(0.80)

Находим выбросы, у которых квантиль менее 10% и более 90% и отбрасываем их

In [ ]:
def find_outburst(column): #На входе передаём название колонки с выбросами, которые будут исключены
    low_fence = df[column].quantile(0.10)
    high_fence = df[column].quantile(0.90)
    df_out = df.loc[(df[column] > low_fence) & (df[column] < high_fence)]
    return df_out

In [ ]:
df_out = find_outburst('Age')

In [ ]:
sns.boxplot(x=df_out['Age'])

2 - Создание признака исключения для значений-выбросов

In [59]:
def new_feat_outburst(column): #На входе передаём название колонки с выбросами, по которым будет создан признак
    low_fence = df[column].quantile(0.10)
    high_fence = df[column].quantile(0.90)
    df['Outburst'] = np.where((df[column] > low_fence) & (df[column] < high_fence), 0, 1)
    return df

# Категориальные признаки

**Полезные ссылки** 
[категориальные признаки](https://dyakonov.org/2016/08/03/python-%D0%BA%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D0%B5-%D0%BF%D1%80%D0%B8%D0%B7%D0%BD%D0%B0%D0%BA%D0%B8/)

**Нахождение категориальных признаков**

In [ ]:
def find_cat(df): #На входе передаём датафрейм
    for name in df.columns:
        cat_feat = ''
        cat_feat += name
        if (type(df[name][0])== str): #Проверяем, что в элементе 0 у столбца строка
            cat_feat += ' строка'
        if (df[name].nunique()<=10): #Находим количество уникальных значений в датафрейме, число 10 изменяем при необходимости - это поможет выбрать категориальные значения
            cat_feat += ' менее 10 уникальных значений'
        if (cat_feat!=name):
            print(cat_feat) #Получаем уникальные значения в колонках по типу указанному в функции
            
find_cat(df)

**Создаём новые категориальные признаки**

In [ ]:
# Объединение 2ух значений (конъюнция)
def make_feat_conj(df, cat1, cat2): #на входе передаём название датафрейма и названия колонок, над которыми совершается преобразование
    df[cat1 + '+' + cat2] = df[cat1].astype(str) + '+' + df[cat2].astype(str)
    return df
make_feat_conj(df, 'Sex', 'Embarked')

Есть простейший кодировщий sklearn.preprocessing.LabelEncoder, который каждой категории сопоставляет некоторое целое число (собственно, номер категории). Даже если бы его не было, то такую кодировку несложно написать самому с помощью функции map. Для этого предварительно задаётся словарь, в котором указывается, что и чем кодировать.


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(df.Sex)
df['Sex_le'] = le.transform(df.Sex)
df

Ручное кодирование через map

In [ ]:
dct = {'male': 0, 'female': 1}

df['Sex_le'] = df['Sex'].map(dct)
df

**Dummy-кодирование**

Есть простейший метод кодирования, его часто называют наивным / глупым (dummy) кодированием или one-hot-кодированием. Для кодируемого категориального признака создаются N новых признаков, где N — число категорий. Каждый i-й новый признак — бинарный характеристический признак i-й категории.

Предположим, что некоторый признак может принимать 10 разных значений. В этом случае OneHot подразумевает создание 10 признаков, все из которых равны нулю за исключением одного. На позицию, соответствующую численному значению признака мы помещаем 1.

In [ ]:
#Работает только с целочисленными значениями
from sklearn.preprocessing import OneHotEncoder
just_dummies = pd.get_dummies(df['Embarked'])
#где df['dummy'] - столбец, который нужно закодировать
#Конкатенируешь к датафрейму df закодированные столбцы
step_1 = pd.concat([df, just_dummies], axis=1)      
#Удаляешь столбец на основе которого строил признаки
step_1.drop(['Embarked'], inplace=True, axis=1)
df

Среднее значение 2ух столбцов

In [ ]:
def code_mean(df, cat_feat, real_feat): #на входе передаём название датафрейма и названия колонок, над которыми совершается преобразование
    return(df[cat_feat].map(df.groupby(cat_feat)[real_feat].mean()))

df['Sex_Age_mean'] = code_mean(df, 'Sex', 'Age')
df

**Хэширование признаков**

Реальные данные могут оказаться гораздо более динамичными, и мы не всегда можем рассчитывать, что категориальные признаки не будут принимать новых значений. Все это сильно затрудняет использование уже обученных моделей на новых данных. Кроме того, LabelEncoder подразумевает предварительный анализ всей выборки и хранение построенных отображений в памяти, что затрудняет работу в режиме больших данных.

Для решения этих проблем существует более простой подход к векторизации категориальных признаков, основанный на хэшировании.

Хэш-функции могут помочь нам в задаче поиска уникальных кодов для различных значений признака.
На примере с полом пассажира это может выглядеть так:

In [41]:
# Отрицательные и большие по модулю значения нам не подойдут, ограничим область значений хэш-функции:
hash_space = 25

for s in ('female', 'male'): 
    print(s, '->', hash(s) % hash_space)

female -> 23
male -> 1


С помощью хэширования, к примеру, работают некоторые облачные вычислительные сервера типа Kaggle. В них некоторым образом анализируется исходный код программ, вычисляется хэш и подбирается наиболее подходящий сервер для работы конкретной программы.

# Нормализация признаков

**Шкалирование признаков**

In [56]:
def scaler(scaler_type, column1, column2): #На вход подаётся строка, тип scaler = Robust или Standart и 2 колонки в датафрейме
    x = df[[column1, column2]]
    if scaler_type=='Robust':
        # Создать шкалировщик
        scaler = preprocessing.RobustScaler()
        # Преобразовать признак
        standardized = scaler.fit_transform (x)
    if scaler_type=='Standart':
        # Создать шкалировщик
        scaler = preprocessing.StandardScaler()
        # Преобразовать признак
        standardized = scaler.fit_transform (x)
    return standardized #на выходе получаем стандартизированную колонку
scaler('Robust', 'Age', 'Pclass')